# 03: Black-Litterman vs Monte Carlo Diagnostics

This notebook demonstrates, step by step, how to align the Black-Litterman and Monte Carlo optimizers by calibrating risk aversion, view strength, constraints, and risk-free rate treatment.

In [1]:
import pandas as pd
import numpy as np
from quantfolio_engine.optimizer.black_litterman import BlackLittermanOptimizer
from quantfolio_engine.optimizer.monte_carlo import MonteCarloOptimizer
from quantfolio_engine.config import PROCESSED_DATA_DIR, ASSET_UNIVERSE
import matplotlib.pyplot as plt

# Load data
returns = pd.read_csv(PROCESSED_DATA_DIR / 'returns_monthly.csv', index_col=0, parse_dates=True)
macro = pd.read_csv(PROCESSED_DATA_DIR / 'macro_monthly.csv', index_col=0, parse_dates=True)
exposures = pd.read_csv(PROCESSED_DATA_DIR / 'factor_exposures.csv', index_col=0, parse_dates=True)
regimes = pd.read_csv(PROCESSED_DATA_DIR / 'factor_regimes_hmm.csv', index_col=0, parse_dates=True)
assets = list(ASSET_UNIVERSE.keys())
returns = returns[assets]


2025-07-04 18:28:28.968 | INFO     | quantfolio_engine.config:<module>:12 - PROJ_ROOT path is: /Users/dominusdeorum/Documents/Vanderbilt/Projects/quantfolio-engine


## 1. Calibrate λ for Black-Litterman
Find λ so that annualized mean(π) slightly exceeds rf.

In [2]:
rf_annual = 0.045
rf_monthly = rf_annual / 12
cov = returns.cov()
w_mkt = np.ones(len(assets)) / len(assets)
for lam in [0.05, 0.08, 0.10, 0.15, 0.20, 0.25, 0.30]:
    pi = lam * cov.values @ w_mkt
    print(f'λ={lam:.2f}, mean(π)={pi.mean()*12:.4%} p.a., rf={rf_annual:.2%}')

λ=0.05, mean(π)=0.0928% p.a., rf=4.50%
λ=0.08, mean(π)=0.1484% p.a., rf=4.50%
λ=0.10, mean(π)=0.1856% p.a., rf=4.50%
λ=0.15, mean(π)=0.2783% p.a., rf=4.50%
λ=0.20, mean(π)=0.3711% p.a., rf=4.50%
λ=0.25, mean(π)=0.4639% p.a., rf=4.50%
λ=0.30, mean(π)=0.5567% p.a., rf=4.50%


## 2. Boost view returns/strength
Double the base view returns or set view_strength=3–4. Show effect on posterior μ and weights.

In [3]:
bl = BlackLittermanOptimizer(risk_free_rate=rf_annual, lambda_mkt=0.10)  # Use λ from above
cov = returns.cov()
pi = bl.calculate_equilibrium_returns(cov)
P, Q, Omega = bl.create_factor_timing_views(exposures, regimes, returns, view_strength=3.0)
print('Mean Q (monthly):', np.mean(Q))
print('Mean π (monthly):', pi.mean())
# Posterior
tau_sigma = bl.tau * cov.values
M1 = np.linalg.inv(tau_sigma)
M2 = P.T @ np.linalg.inv(Omega) @ P
M = M1 + M2
m1 = M1 @ pi.values
m2 = P.T @ np.linalg.inv(Omega) @ Q
mu_bl = np.linalg.inv(M) @ (m1 + m2)
print('Posterior μ (monthly):', mu_bl)
print('Posterior μ (annual):', mu_bl*12)


2025-07-04 18:28:29.859 | INFO     | quantfolio_engine.optimizer.black_litterman:calculate_equilibrium_returns:142 - Market cap weights: {'SPY': np.float64(0.08333333333333333), 'TLT': np.float64(0.08333333333333333), 'GLD': np.float64(0.08333333333333333), 'AAPL': np.float64(0.08333333333333333), 'MSFT': np.float64(0.08333333333333333), 'JPM': np.float64(0.08333333333333333), 'UNH': np.float64(0.08333333333333333), 'WMT': np.float64(0.08333333333333333), 'XLE': np.float64(0.08333333333333333), 'BA': np.float64(0.08333333333333333), 'IWM': np.float64(0.08333333333333333), 'EFA': np.float64(0.08333333333333333)}
2025-07-04 18:28:29.874 | INFO     | quantfolio_engine.optimizer.black_litterman:calculate_equilibrium_returns:143 - Lambda market: 0.1
2025-07-04 18:28:29.875 | INFO     | quantfolio_engine.optimizer.black_litterman:calculate_equilibrium_returns:144 - Equilibrium returns (monthly): {'SPY': 0.00016391536516665098, 'TLT': 2.2722002476780584e-05, 'GLD': 2.010381562853374e-05, 'AAP

## 3. Add a 'grand view' using Monte Carlo mean as a BL view
Add a view: 'The equal-weight portfolio will return X% per month.' Show effect.

In [4]:
mc_mean = returns.mean(axis=1).mean()
grand_P = np.ones((1, len(assets))) / len(assets)
grand_Q = np.array([mc_mean])
grand_Omega = np.array([[0.0001]])  # Small uncertainty
# Combine with previous views
P2 = np.vstack([P, grand_P])
Q2 = np.concatenate([Q, grand_Q])
Omega2 = np.block([
    [Omega, np.zeros((Omega.shape[0], 1))],
    [np.zeros((1, Omega.shape[0])), grand_Omega]
])
M1 = np.linalg.inv(tau_sigma)
M2 = P2.T @ np.linalg.inv(Omega2) @ P2
M = M1 + M2
m1 = M1 @ pi.values
m2 = P2.T @ np.linalg.inv(Omega2) @ Q2
mu_bl2 = np.linalg.inv(M) @ (m1 + m2)
print('Posterior μ with grand view (annual):', mu_bl2*12)


Posterior μ with grand view (annual): [ 0.02895806  0.04683708  0.02797559  0.04547313 -0.02677143  0.0201681
  0.05071998  0.06608159  0.03593005  0.02017794  0.04369731  0.02550707]


## 4. Relax weight caps
Try max_weight=0.4, min_weight=0.0 and show optimizer's preference.

In [5]:
bl = BlackLittermanOptimizer(risk_free_rate=rf_annual, lambda_mkt=0.10)
results = bl.optimize_portfolio(
    returns_df=returns,
    factor_exposures=exposures,
    factor_regimes=regimes,
    constraints={'max_weight': 0.4, 'min_weight': 0.0}
)
print('Weights:', results['weights'])
print('Expected return (annual):', results['expected_return'])
print('Sharpe ratio:', results['sharpe_ratio'])


2025-07-04 18:28:29.893 | INFO     | quantfolio_engine.optimizer.black_litterman:optimize_portfolio:444 - Starting Black-Litterman portfolio optimization...
2025-07-04 18:28:29.893 | INFO     | quantfolio_engine.optimizer.black_litterman:estimate_covariance_matrix:81 - Estimating covariance matrix using sample method...
2025-07-04 18:28:29.894 | INFO     | quantfolio_engine.optimizer.black_litterman:estimate_covariance_matrix:109 - Covariance matrix shape: (12, 12)
2025-07-04 18:28:29.894 | INFO     | quantfolio_engine.optimizer.black_litterman:calculate_equilibrium_returns:142 - Market cap weights: {'SPY': np.float64(0.08333333333333333), 'TLT': np.float64(0.08333333333333333), 'GLD': np.float64(0.08333333333333333), 'AAPL': np.float64(0.08333333333333333), 'MSFT': np.float64(0.08333333333333333), 'JPM': np.float64(0.08333333333333333), 'UNH': np.float64(0.08333333333333333), 'WMT': np.float64(0.08333333333333333), 'XLE': np.float64(0.08333333333333333), 'BA': np.float64(0.08333333333

## 5. Make rf treatment consistent in both engines
Subtract rf in Monte Carlo objective, or set rf=0 in both. Compare Sharpe ratios.

In [6]:
# Monte Carlo with rf subtracted
mc = MonteCarloOptimizer(risk_free_rate=rf_annual)
mc_results = mc.optimize_with_constraints(
    scenarios=mc.generate_scenarios(returns),
    constraints={'max_weight': 0.4, 'min_weight': 0.0}
)
print('Monte Carlo expected return (annual):', mc_results['expected_return'])
print('Monte Carlo Sharpe ratio:', mc_results['sharpe_ratio'])


2025-07-04 18:28:30.002 | INFO     | quantfolio_engine.optimizer.monte_carlo:generate_scenarios:79 - Generating 1000 Monte Carlo scenarios...
2025-07-04 18:28:30.004 | INFO     | quantfolio_engine.optimizer.monte_carlo:generate_scenarios:95 - Generated scenarios with shape: (1000, 12, 12)
2025-07-04 18:28:30.004 | INFO     | quantfolio_engine.optimizer.monte_carlo:optimize_with_constraints:121 - Optimizing portfolio with Monte Carlo constraints...
2025-07-04 18:28:30.005 | INFO     | quantfolio_engine.optimizer.monte_carlo:optimize_with_constraints:136 - Annualizing mean and covariance with factor 12.00
2025-07-04 18:28:30.045 | SUCCESS  | quantfolio_engine.optimizer.monte_carlo:optimize_with_constraints:219 - Monte Carlo optimization completed. Sharpe: 1.118
Monte Carlo expected return (annual): 0.2524463511740621
Monte Carlo Sharpe ratio: 1.118399274498926


## 6. Final comparison: Aligned assumptions
Show that, after aligning λ, rf, views, and constraints, the two engines converge to similar risk-adjusted performance.

In [7]:
print('Black-Litterman Sharpe:', results['sharpe_ratio'])
print('Monte Carlo Sharpe:', mc_results['sharpe_ratio'])


Black-Litterman Sharpe: -0.24481241665288037
Monte Carlo Sharpe: 1.118399274498926
